In [ ]:
import pandas as pd
import re

police_path = "울릉경찰서 교통계(교통사고)_2025년도.csv"
ulleung_path = "ulleung_accidents_with_coords.csv"

police = pd.read_csv(police_path)
ulleung = pd.read_csv(ulleung_path)

place_col = [c for c in police.columns if "사고" in c and "장소" in c][0]
type_col = [c for c in police.columns if "종별" in c][0]


def normalize(text):
    if pd.isna(text):
        return ""
    text = str(text).replace("
", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

police = police[[place_col, type_col]].copy()
police[place_col] = police[place_col].map(normalize)
police[type_col] = police[type_col].map(normalize)
police = police[(police[place_col] != "") & (police[type_col] != "")]

police = police.drop_duplicates(subset=[place_col])

mapping = dict(zip(police[place_col], police[type_col]))

ulleung["type"] = ulleung["raw"].map(lambda s: mapping.get(normalize(s), pd.NA))
ulleung.to_csv(ulleung_path, index=False)

missing = ulleung[ulleung["type"].isna()][["raw"]]
print("missing count", len(missing))
print(missing.to_string(index=False))
